In [1]:
import pandas as pd



df_events = pd.read_csv("../data/events.csv")
df_items = pd.read_csv("../data/df_item_clean.csv")


In [2]:
df_events['event'].value_counts(normalize=True)


event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64

In [ ]:
# Nettoyage des données

df_events.isna().sum()
df_events = df_events.dropna(subset=['itemid'])





timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64


In [ ]:
df_merged = df_events.merge(
    df_items,
    on='itemid',
    how='left'
)



df_merged.shape

df_merged.isna().mean().sort_values(ascending=False)




transactionid    0.992521
parentid         0.043358
value            0.043318
categoryid       0.043318
property         0.043318
timestamp_y      0.043318
visitorid        0.000000
timestamp_x      0.000000
event            0.000000
itemid           0.000000
dtype: float64

In [18]:
# sauvegarde de df_merged
df_merged.to_csv("../data/df_merged.csv", index=False)

In [9]:
# les evenements par itemid
item_events = (
    df_merged
    .groupby(['itemid', 'event'])
    .size()
    .unstack(fill_value=0)
)

print(item_events.head())

event   addtocart  transaction  view
itemid                              
3               0            0     2
4               0            0     3
6               0            0    29
9               0            0     2
15              3            1    18


In [10]:
# 🔹 Volumes par étape

funnel = df_merged['event'].value_counts()
funnel

event
view           5706279
addtocart       149857
transaction      44127
Name: count, dtype: int64

In [11]:
# 🔹 Funnel par utilisateur unique

df_merged.groupby('event')['visitorid'].nunique()


event
addtocart        37722
transaction      11719
view           1404179
Name: visitorid, dtype: int64

In [12]:
# Taux de conversion

views = funnel.get('view', 0)
add_to_cart = funnel.get('addtocart', 0)
transactions = funnel.get('transaction', 0)

add_to_cart_rate = add_to_cart / views
purchase_rate = transactions / views

add_to_cart_rate, purchase_rate


(np.float64(0.026261772338856897), np.float64(0.007733060370865147))

In [ ]:
# Analyse par produit

In [13]:
# Produits les plus vus

df_merged[df_merged['event'] == 'view'] \
    .groupby('itemid') \
    .size() \
    .sort_values(ascending=False) \
    .head(10)


itemid
320130    23994
354233    17694
133814    15732
247909    14832
158666    13752
359491    13248
15948     12798
291877    11268
316779    10890
378637     9810
dtype: int64

In [14]:
# Produits les plus achetés

df_merged[df_merged['event'] == 'transaction'] \
    .groupby('itemid') \
    .size() \
    .sort_values(ascending=False) \
    .head(10)


itemid
320130    594
46156     558
316472    252
210087    216
210002    162
158666    162
390591    144
19883     144
461686    133
388536    126
dtype: int64

In [15]:
# Analyse par catégorie
df_merged.groupby('categoryid')['event'] \
    .value_counts() \
    .unstack() \
    .fillna(0)


event,addtocart,transaction,view
categoryid,,,
0.0,59.0,12.0,1113.0
1.0,392.0,80.0,5847.0
2.0,48.0,14.0,4304.0
3.0,58.0,15.0,2375.0
4.0,13.0,10.0,144.0
...,...,...,...
1689.0,0.0,0.0,19.0
1690.0,29.0,5.0,1351.0
1694.0,94.0,44.0,5557.0


In [ ]:
# Analyse du comportement utilisateur

In [16]:
# Nombre d’événements par utilisateur

df_merged.groupby('visitorid').size().describe()


count    1.407580e+06
mean     4.191778e+00
std      2.891248e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      1.375100e+04
dtype: float64

In [17]:
# Utilisateurs acheteurs vs non-acheteurs

buyers = df_merged[df_merged['event'] == 'transaction']['visitorid'].nunique()
total_users = df_merged['visitorid'].nunique()

buyers / total_users


0.00832563690873698